In [14]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image

from numpy import load
from numpy import expand_dims
from numpy import savez_compressed
from keras.models import load_model

from tqdm import tqdm
from tqdm import tqdm_notebook

import json

import matplotlib.pyplot as plt
%matplotlib inline

import os.path
from os import path

import warnings
warnings.filterwarnings('ignore')

In [2]:
import cv2
from cv2 import CascadeClassifier
classifier = CascadeClassifier('cascade_models/haarcascade_frontalface_default.xml')

## The Main DataFrames:

In [3]:
#Load the converted metadata files for the images
photo_df = pd.read_csv('photo_info_df/All_photo_data.csv')

In [4]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766


In [5]:
#There are 372917 total images across 53690 individuals
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372917 entries, 0 to 372916
Data columns (total 8 columns):
name              372917 non-null object
dob               372917 non-null object
gender            372917 non-null float64
photo_taken       372917 non-null int64
age_when_taken    372917 non-null int64
file_path         372917 non-null object
face_location     372917 non-null object
face_score        372917 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 22.8+ MB


In [6]:
def obtain_image_pixels(filename):
    image = Image.open(filename)
    image = image.convert('RGB')
    return asarray(image)

In [ ]:
#photo_df[photo_df['file_path'].str.contains("image_data/imdb_data/", case=False)]

In [7]:
def get_all_faces(image_array):
    
    #Obtain the first detected face in the cropped face picture
    faces_detected = classifier.detectMultiScale(image_array)
    
    #MTCNN already returns the list by sorted confidence level
    return faces_detected

In [8]:
def resize_picture(image_array, face_box, dimensions = (160,160), margin = 8):
            
    #Set a margin boolean and while loop to try margin value
    margin_error = True
    
    while margin_error:
    
        try:
        
            # get coordinates
            x1, y1, width, height = face_box
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [9]:
def get_all_resized_faces(file_path_name, dimensions = (160,160), margin = 8):
    
    all_image_pixels = obtain_image_pixels(file_path_name)
    
    all_image_faces = get_all_faces(all_image_pixels)
    
    face_array_list = []
    
    #The reason I return the entire image is because the main dataset already lists minimum
    #detected confidence level by face, I remove certain images by this value later
    if len(all_image_faces) == 0:
        
        face_array_resized = Image.fromarray(all_image_pixels)
        face_array_resized = face_array_resized.resize(dimensions)
        face_array_list.append(asarray(face_array_resized))
        
    else:
        for i in all_image_faces:
            face_array_list.append(resize_picture(all_image_pixels, i, dimensions, margin))
            
    return face_array_list

In [10]:
def plot_face(face_data, is_array = True):
    
    #A use case if the array is sent
    if is_array:
        # plot face
        plt.axis('off')
        plt.imshow(face_data)
        plt.show()
    
    #A use case if a file path is sent
    else:
        # load image from file as array
        pixels = plt.imread(face_data)

        #Display image unedited
        plt.axis('off')
        plt.imshow(pixels)
        plt.show()

In [15]:
#A check to see if the file path exists
file_exists = 0
file_not_exist = 0

file_not_exist_list = []

for i in photo_df.file_path:
    if path.exists(i):
        file_exists += 1
    else: 
        
        file_not_exist_list.append(i)
        file_not_exist += 1
        
        
print(file_exists)
print(file_not_exist)

187662
185255


In [18]:
ls

CNN Basic.ipynb                       fred_image/
EDA.ipynb                             functioning functions/
FaceNet Base Model.ipynb              image_data/
FaceNet Model-Friday.ipynb            imdb_metadata_cleanup.ipynb
FaceNet Model.ipynb                   move_relevant_images_notebook.ipynb
FaceNet_Model/                        photo_info_df/
Image To Array Notebook-Copy1.ipynb   photo_metadata.csv
Image To Array Notebook.ipynb         photo_metadata_second_faces.csv
README.md                             wiki_metadata_cleanup.ipynb
cascade_models/                       wiki_photo_metadata.csv
config.py                             wiki_photo_metadata_second_faces.csv
first_1000_wiki


In [17]:
len(file_not_exist_list)

185255

In [12]:
tqdm.pandas()

photo_df['face_pixels'] = photo_df.file_path.progress_apply(lambda x: get_all_resized_faces(x, dimensions = (160,160))[0])
photo_df.to_csv('photo_info_df/photo_data_with_image_arrays_column.csv', index = False)

  0%|          | 368/372917 [00:16<7:08:01, 14.51it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'image_data/imdb_data/36/nm0000036_rm317635072_1895-10-4_1965.jpg'

  0%|          | 369/372917 [00:30<7:08:00, 14.51it/s]

In [ ]:
photo_df_A = photo_df[:50000]
photo_df_B = photo_df[50000:100000]
photo_df_C = photo_df[100000:150000]
photo_df_D = photo_df[150000:200000]
photo_df_E = photo_df[200000:250000]
photo_df_F = photo_df[250000:300000]
photo_df_G = photo_df[300000:350000]
photo_df_H = photo_df[350000:]

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_A['original_image_array'] = photo_df_A['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_A.to_json('photo_info_df/photo_data_with_image_arrays_A.json')

In [ ]:
photo_df_A = pd.read_json('photo_info_df/photo_data_with_image_arrays_A.json')
photo_df_A.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_B['original_image_array'] = photo_df_B['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_B.to_json('photo_info_df/photo_data_with_image_arrays_B.json', index = False)

In [ ]:
photo_df_B = pd.read_json('photo_info_df/photo_data_with_image_arrays_B.json', index = False)
photo_df_B.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_C['original_image_array'] = photo_df_C['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_C.to_json('photo_info_df/photo_data_with_image_arrays_C.json', index = False)

In [ ]:
photo_df_C = pd.read_json('photo_info_df/photo_data_with_image_arrays_C.json', index = False)
photo_df_C.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_D['original_image_array'] = photo_df_D['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_D.to_json('photo_info_df/photo_data_with_image_arrays_D.json', index = False)

In [ ]:
photo_df_D = pd.read_json('photo_info_df/photo_data_with_image_arrays_D.json', index = False)
photo_df_D.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_E['original_image_array'] = photo_df_E['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_E.to_json('photo_info_df/photo_data_with_image_arrays_E.json', index = False)

In [ ]:
photo_df_E = pd.read_json('photo_info_df/photo_data_with_image_arrays_E.json', index = False)
photo_df_E.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_F['original_image_array'] = photo_df_F['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_F.to_json('photo_info_df/photo_data_with_image_arrays_F.json', index = False)

In [ ]:
photo_df_F = pd.read_json('photo_info_df/photo_data_with_image_arrays_F.json', index = False)
photo_df_F.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_G['original_image_array'] = photo_df_G['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_G.to_json('photo_info_df/photo_data_with_image_arrays_G.json', index = False)

In [ ]:
photo_df_G = pd.read_json('photo_info_df/photo_data_with_image_arrays_G.json', index = False)
photo_df_G.head()

In [ ]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_H['original_image_array'] = photo_df_H['file_path'].progress_apply(lambda x: obtain_image_array(x))


In [ ]:
photo_df_H.to_json('photo_info_df/photo_data_with_image_arrays_H.json', index = False)

In [ ]:
photo_df_H = pd.read_json('photo_info_df/photo_data_with_image_arrays_H.json', index = False)
photo_df_H.head()

In [ ]:
#This block combines and saves the dataframes
photo_df = pd.concat([photo_df_A, photo_df_B, photo_df_C, photo_df_D]).reset_index(drop=True)
photo_df.to_csv('photo_info_df/photo_data_with_image_arrays.csv', index = False)

In [ ]:
photo_df.head()

In [ ]:
photo_df.file_path[7]

In [ ]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    
    #Set a margin boolean and while loop to try margin value
    margin_error = True
    
    while margin_error:
    
        try:
            #Crop the face further with MTCNN
            detector = MTCNN()
    
            #Obtain the first detected face in the cropped face picture
            first_detected_face = detector.detect_faces(image_array)[0]
        
            # get coordinates
            x1, y1, width, height = first_detected_face['box']
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
        except:
        
            face_array = image_array
        
        try:
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [ ]:
a = resize_picture('image_data/imdb_data/02/nm0000002_rm1363385088_1924-9-16_2004.jpg', dimensions = (160,160), margin = 0)

In [ ]:
plt.imshow(a)

In [ ]:
picture1 = resize_picture(photo_df.file_path[0], dimensions = (160,160), margin = 10)

In [ ]:
def plot_face(face_data):
    
    # plot face
    plt.axis('off')
    plt.imshow(face_data)

In [ ]:
plot_face(picture1)

In [ ]:
#photo_df[photo_df['file_path'].str.contains("961400_1953-02-05_2011", case=False)]
#photo_df[photo_df['face_score']<=1]
photo_df.file_path[photo_df['face_score'] == photo_df['face_score'].max()]
photo_df.file_path[54153]

In [ ]:
test = photo_df[:100].copy()

In [ ]:
#tqdm.pandas()
#test['original_image_array'] = test['file_path'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))



In [ ]:
tqdm.pandas()
test['original_image_array'] = test['file_path'].progress_apply(lambda x: obtain_image_array(x))



In [ ]:
tqdm.pandas()
test['fn_resized_array'] = test['original_image_array'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))



In [ ]:
test.fn_resized_array[3].shape

### Preparation For The Model

In [ ]:
#The following function is from 'Deep Learning for Computer Vision' by Jason Brownlee, Page (508)

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

### The Model

In [ ]:
# load the FaceNet model
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')
print(facenet_model.inputs)
print(facenet_model.outputs)

In [ ]:
get_embedding(facenet_model, picture1)